In [20]:
import json

with open('design.out.json', 'r') as f:
    design_out_raw = json.load(f)

In [21]:
from dasi.design.output import filter_output_by_design_keys
from typing import *
from copy import deepcopy
    
    
query_keys = []
for key, design in design_out['designs'].items():
    assemblies = design['assemblies']
    if assemblies:
        a = assemblies[0]
        complexity = a['cost']['max synthesis complexity']
        if complexity < 10.0:
            query_keys.append(key)
            
def _filter_dict(data, keys):
    return {k: deepcopy(v) for k, v in data.items() if k in keys}


def filter_output_by_design_keys(
    output, keys, only_assemblies: Optional[List[str]] = None
):
    new_out = {}

    new_out["metadata"] = deepcopy(output["metadata"])
    new_out["designs"] = _filter_dict(output["designs"], keys)

    rids = []
    for design in new_out["designs"].values():
        for assembly in design["assemblies"]:
            for reaction in assembly["summary"]:
                rids.append(reaction["reaction_index"])
            rids += assembly['final_assembly_reaction']
    rids = sorted(list(set(rids)))
    
    print(rids)
    
    rdict = {r["__index__"]: r for r in output["reactions"]}
    mdict = {r["__index__"]: r for r in output["molecules"]}
    reactions = [rdict[r] for r in rids]

    mids = []
    for r in reactions:
        mids += r["inputs"]
        mids += r["outputs"]
    mids = sorted(list(set(mids)))

    molecules = [mdict[m] for m in mids]

    new_out["reactions"] = reactions
    new_out["molecules"] = molecules
    return new_out

design_out = deepcopy(design_out_raw)

print(design_out.keys())

dict_keys(['metadata', 'designs', 'molecules', 'reactions'])


In [22]:
design = list(design_out['designs'].values())[0]
assembly = design['assemblies'][0]
design.keys()

dict_keys(['compiled', 'run', 'success', 'assemblies', 'record', 'sequence'])

In [12]:
assembly

{'cost': {'material cost': 773.2,
  'assembly efficiency': 0.01,
  'max synthesis complexity': 36.29},
 'summary': [{'reaction_index': 7,
   'metadata': {'cost': 10.526315789473685,
    'materials': 10.526315789473685,
    'efficiency': 0.95,
    'complexity': None},
   'outputs': [{'start': 4, 'end': 263, 'type': 'PCR_PRODUCT_WITH_PRIMERS'}]},
  {'reaction_index': 50,
   'metadata': {'cost': 492.59259259259255,
    'materials': 492.59259259259255,
    'efficiency': 0.81,
    'complexity': 1.837325905292479},
   'outputs': [{'start': 233, 'end': 2093, 'type': 'GAP'}]},
  {'reaction_index': 51,
   'metadata': {'cost': 10.526315789473685,
    'materials': 10.526315789473685,
    'efficiency': 0.95,
    'complexity': None},
   'outputs': [{'start': 2058,
     'end': 3194,
     'type': 'PCR_PRODUCT_WITH_PRIMERS'}]},
  {'reaction_index': 57,
   'metadata': {'cost': 10.526315789473685,
    'materials': 10.526315789473685,
    'efficiency': 0.95,
    'complexity': None},
   'outputs': [{'star

In [13]:
r = design_out['reactions'][0]
r

{'__name__': 'Assembly',
 '__index__': 0,
 '__type__': 'reaction',
 'used_in_assemblies': [{'design_key': 'baabe288-8fca-4274-af42-b85684d9ac09',
   'assembly': 0}],
 'inputs': [53, 54, 28, 38, 46, 29, 4],
 'outputs': [0],
 'metadata': {'material_cost': 699.4,
  'assembly_efficiency': 0.014726337767428706,
  'max_complexity': 36.28855721393035}}

In [15]:
m = design_out['molecules'][0]
m

{'__name__': 'PLASMID',
 '__index__': 0,
 '__type__': 'molecule',
 '__meta__': {},
 'sequence': {'bases': 'AAACATTAGTTATGTCACGCTTACATTCACGCCCTCCCCCCACATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGGCTTTAATTTGATGTCGTAATAACCCCGCCCCGAAAACTGTATTATAAGTAAATGCATGTATACTAAACTCACAAATTAGAGCTTCAATTTAATTATATCAGTTATTACCCGGGAATCTCGGTCGTAATGATTTCTATAATGACGAAAAAAAAAAAATTGGAAAGAAAAAGCTTCATGGCCTTTATAAAAAGGAACTATCCAATACCTCGCCAGAACCAAGTAACAGTATTTTACGGGGCACAAATCAAGAACAATAAGACAGGACTGTAAAGATGGACGCATTGAACTCCAAAGAACAACAAGAGTTCCAAAAAGTAGTGGAACAAAAGCAAATGAAGGATTTCATGCGTTTGTACTCTAATCTGGTAGAAAGATGTTTCACAGACTGTGTCAATGACTTCACAACATCAAAGCTAACCAATAAGGAACAAACATGCATCATGAAGTGCTCAGAAAAGTTCTTGAAGCATAGCGAACGTGTAGGGCAGCGTTTCCAAGAACAAAACGCTGCCTTGGGACAAGGCTTGGGCCGGTTTAAACCATGGTCATAGCTGTTTCCTGTGTGAAATTGTTATCCGCTCACAATTCCACACAACATACGAGCCGGAAGCATAAAGTGTAAAGCCTGGGGTGCCTAATGAGTGAGGTAACTCACATTA

In [80]:
import json
from copy import deepcopy

with open('design.out.json', 'r') as f:
    design_out = json.load(f)
design_out_orig = deepcopy(design_out)

import networkx as nx
from typing import *

def out_to_dag(out: Dict) -> nx.DiGraph:
    """Create a DAG from the output json"""
    g = nx.DiGraph()
    g.add_node('root', data=out['metadata'])
    for m in out['molecules']:
        g.add_node(('molecule', m['__index__']), data=m)
    for r in out['reactions']:
        g.add_node(('reaction', r['__index__']), data=r)
        for i in r['inputs']:
            g.add_edge(('molecule', i), ('reaction', r['__index__']))
        for o in r['outputs']:
            g.add_edge(('reaction', r['__index__']), ('molecule', o))
    for qk, d in out['designs'].items():
        g.add_node(('design', qk), data=d)
        g.add_edge('root', ('design', qk))
        for ai, a in enumerate(d['assemblies']):
            g.add_edge(('design', qk), ('assembly', ai, qk))
            g.add_node(('assembly', ai, qk), data=a)
            final_index = a['final_assembly_reaction'][0]
            g.add_edge(('assembly', ai, qk), ('reaction', final_index))
    return g


gout = out_to_dag(design_out)
print(nx.info(gout))
tree = nx.dfs_tree(gout, 'root')
print(nx.info(tree))

for n in gout.nodes():
    if n[0] == 'molecule':
        if not list(gout.predecessors(n)):
            if gout.nodes[n]['data']['__name__']
                gout.nodes[n]['data']['__type__']['']
                print(str(n) + " has no predecessors")

            
# gout.nodes[('molecule', 53)]
# # gout.nodes[('reaction'), 67]

# # gout[('reaction', 67)]

# # list(gout.predecessors(('molecule', 52)))
# list(gout.successors(('reaction', 67)))
# # gout.nodes()
# # for n1, n2 in tree.edges():
# #     print(n1, n2)

gout.nodes[('molecule', 22)]

SyntaxError: invalid syntax (<ipython-input-80-2f91623e8390>, line 42)

In [69]:
design_out['reactions'][67]

{'__name__': 'Synthesize',
 '__index__': 67,
 '__type__': 'reaction',
 'used_in_assemblies': [{'design_key': 'baabe288-8fca-4274-af42-b85684d9ac09',
   'assembly': 0}],
 'inputs': [],
 'outputs': [53],
 'metadata': {'material_cost': 121.4,
  'efficiency': 0.027556896078969013,
  'complexity': 36.28855721393035}}